# Mount the Gdrive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!ls "/content/gdrive/My Drive/eva4p2/drone-dataset1.zip"

'/content/gdrive/My Drive/eva4p2/drone-dataset1.zip'


#Copy the Folder to Colab for Training

In [4]:
cd /content

/content


In [5]:
!ls

gdrive	sample_data


In [6]:
cp -r "/content/gdrive/My Drive/eva4p2/drone-dataset1.zip" "raw-drone-dataset.zip"

In [7]:
!unzip raw-drone-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: drone-dataset/train/3/Image_67.jpg  
  inflating: drone-dataset/train/3/sandhu-eva4p2-imgg0174.jpg  
  inflating: drone-dataset/train/3/wing_Drone_0_B _95_.jpg  
  inflating: drone-dataset/train/3/capture1.jpg  
  inflating: drone-dataset/train/3/dsc07909.jpg  
  inflating: drone-dataset/train/3/image221.jpg  
  inflating: drone-dataset/train/3/217095_web.jpg  
  inflating: drone-dataset/train/3/5e31b98be0.jpg  
  inflating: drone-dataset/train/3/Image_316.jpg  
  inflating: drone-dataset/train/3/Image_417.jpg  
  inflating: drone-dataset/train/3/Image_692.jpg  
  inflating: drone-dataset/train/3/Tellus4D-2.jpg  
  inflating: drone-dataset/train/3/Image_144.jpg  
  inflating: drone-dataset/train/3/Image_293.jpg  
  inflating: drone-dataset/train/3/SkyWalker-X8-Heliguy-crop(1).jpg  
  inflating: drone-dataset/train/3/685080fc37bf3b1a28f92578413146d5.jpg  
  inflating: drone-dataset/train/3/dt26-open-payload-package.jpg  
  

#Preprocess the Files, remove the waste, uncompatible files etc and calculate mean and std 

In [9]:
import numpy as np
import cv2
import io
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms
import zipfile
from PIL import Image
import sys
import torch.utils.data
import timeit
import logging

# This is the common class for both test and train data sets. Based on indexes, this class serves the transformed image tensor.
  
#Considered 4 folders (n). Take all the files from those folders into a list, a global list in memory.     
#Flying birds, Large Quadcopters, Small QuadCopters, Winged drones
#Given an index, list[index] will be returned as a file.
#root_dir = "/content/drone-dataset"
class PreprocessImages(Dataset):

  def __init__(self, root_dir, processed_root_dir, transform=ToTensor()):
    'Initialization'
    self.m_root = root_dir
    self.m_transform = transform
    logging.basicConfig(filename = "/content/prepreprocessing_timeit.log",datefmt='%H:%M:%S',level=logging.DEBUG)
    #self.g_dict_fileindex = dict()
    self.g_array_fileindex1 = []
    self.m_processed_root = processed_root_dir
    self.walkTheDirectory(self.m_root) 
    self.m_length = len(self.g_array_fileindex1)
  
  #root_dir structure
  #train
  #   /folder1
  #     file.jpg
  #     file40.jpg  
  #   /folder2
  #     filex.jpg
  #     filey.40.jpg  
  #   /folder3
  #     filea.jpg
  #     fileb.40.jpg  
  #   /folder4
  #     fileaa.jpg
  #     filebax.jpg  
  
  #test
  #   /anyfoldername1
  #     sfile.jpg
  #     mfile40.jpg  
  #   /foldername2
  #     efilex.jpg
  #     cfiley.40.jpg  
  #   /foldername3
  #     somefilea.jpg
  #     vfileb.40.jpg  
  #   /foldername4
  #     somefileaa.jpg
  #     afilebax.jpg  
  
  
    
  def walkTheDirectory(self,root_dir):
    # for file in files:
    #    with open(os.path.join(root, file), "r") as auto: 
    subdirs = [x[0] for x in os.walk(root_dir)]
    dir = ''
    try:
      dir = self.m_processed_root
      os.mkdir(dir)
    except:
      print('dir present',self.m_processed_root)

    count = -1
    dir1 =  self.m_processed_root
    print('dir1=',dir1) 
    for d in subdirs:
      count = count + 1
      if count == 0:
        continue
      basedir = os.path.basename(subdirs[count])
      dir = os.path.join(dir1,basedir)
      print(dir)
      try:
        os.mkdir(dir)
      except:
        print('dir present', dir)
    
    for path, subdirs, files in os.walk(self.m_root): # path should give the complete path till the last directory.
        for filename in files:
            #print(os.path.join(path, filename))
            full_path = os.path.join(path, filename)
            #self.g_dict_fileindex[index] = full_path
            self.g_array_fileindex1.append(full_path)
            
 
  def __len__(self):
    'Denotes the total number of samples'
    return self.m_length;

  def printItem(self,idx):
    if idx >= self.m_length:
        print('index', idx, 'is bigger than the actual dataset size')
        return
    file = self.g_array_fileindex1[idx]
    print('printFile-',file)  
  
  def removeItem(self,idx):
    if idx >= self.m_length:
        print('index', idx, 'is bigger than the actual dataset size')
        return
    #self.g_array_fileindex.remove(file)

  def __getitem__(self, idx):
    if idx >= self.m_length:
        print('index', idx, 'is bigger than the actual dataset size')
        return
        
    start_time = timeit.default_timer()

    # Pick the file from the index
    size = 224, 224
    file = self.g_array_fileindex1[idx]
    
    file1 = file
    try:
      try:
        image = Image.open(file).convert('RGB') # even grey scale images will be converted to RGB with 3 channels
      except:
        raise "IOError"
      #temporarily we will resize to 224*224
      # We need to look into meaningful way of resizing (...pyramid pooling??)
      width,height = image.size
      #print('width=',width,'height=',height)
      image = image.resize((224,224), Image.BICUBIC)
      #print('image converted with BICUBIC')
      filename, file_extension = os.path.splitext(file)
      basedir = os.path.basename(os.path.dirname(filename)) # 0,1,2,3 dirs of either train or test
      #append subdir to new dir
      dir1 = os.path.join(self.m_processed_root,basedir) # This is new directory path.
    
      #Create jpg file
      basefile=os.path.basename(file)#This comes with extension
      #split with extension
      file1, ext = os.path.splitext(basefile)
      file1 = file1 + ".jpg"
      file1 = os.path.join(dir1,file1)#complete new file path
      #print('new filename=',file1)
      try:
        image.save(file1) # incase if it is already present
        #print('post proprocessed filename=',file1)
      except:
        print('post proprocessed filename=',file1)
        
      if self.m_transform:
        #print('transform to tensor')
        image = self.m_transform(image)
      
      self.g_array_fileindex1[idx] = file1
    except:
      print("exception while opening=",file)
      
    load_time = timeit.default_timer() - start_time
    
    desc = f' file={file1} orig_width={width},orig_height={height}, PROCESSING_TIME={load_time:0.3f}'
    logging.info(desc)
      
    return image


#Calculate Mean and Std of resized files

In [10]:
import torch
from torchvision import transforms

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

#Assuming test data represent the full set of train data. So we use normalized values of traion data in test data also.

train_root_dir = "/content/drone-dataset/train"
processed_train_root_dir = "/content/processed-drone-dataset/train"

tr = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomCrop(),
    transforms.ToTensor()
    #normalize,
    #transforms.RandomErasing()
])


train_data_set = PreprocessImages(train_root_dir, processed_train_root_dir, transform = tr)

def mean_std(dataset):
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                          shuffle=False, num_workers=1)

  mean = torch.zeros(3)
  std = torch.zeros(3)
  print('Computing mean and std from dataloader')
  for inputs in dataloader:
	  for i in range(3):
		  mean[i] = mean[i] + inputs[:,i,:, :].mean()
		  std[i]  = std[i] + inputs[:,i,:, :].std()
  mean.div_(len(dataset))
  std.div_(len(dataset))
  print('mean=',mean)
  print('std=',std)
  return mean, std

print('calculating mean,std of train dataset')
mean_std(train_data_set)

test_root_dir = "/content/drone-dataset/test"
processed_test_root_dir = "/content/processed-drone-dataset/test"

test_data_set = PreprocessImages(test_root_dir, processed_test_root_dir, transform = tr)

print('calculating mean,std of test dataset')
mean_std(test_data_set)


dir present /content/processed-drone-dataset/train
dir1= /content/processed-drone-dataset/train
/content/processed-drone-dataset/train/0
dir present /content/processed-drone-dataset/train/0
/content/processed-drone-dataset/train/3
dir present /content/processed-drone-dataset/train/3
/content/processed-drone-dataset/train/2
dir present /content/processed-drone-dataset/train/2
/content/processed-drone-dataset/train/1
dir present /content/processed-drone-dataset/train/1
calculating mean,std of train dataset
Computing mean and std from dataloader
mean= tensor([0.5266, 0.5789, 0.6107])
std= tensor([0.1701, 0.1641, 0.1792])
dir present /content/processed-drone-dataset/test
dir1= /content/processed-drone-dataset/test
/content/processed-drone-dataset/test/0
dir present /content/processed-drone-dataset/test/0
/content/processed-drone-dataset/test/3
dir present /content/processed-drone-dataset/test/3
/content/processed-drone-dataset/test/2
dir present /content/processed-drone-dataset/test/2
/con

(tensor([0.5261, 0.5786, 0.6074]), tensor([0.1706, 0.1653, 0.1828]))

#Save the Preprocessed dataset as zip file

In [11]:
!zip -r processed-drone-dataset.zip /content/processed-drone-dataset  

Streaming output truncated to the last 5000 lines.
  adding: content/processed-drone-dataset/train/1/Capture161.jpg (deflated 1%)
  adding: content/processed-drone-dataset/train/1/i (14).jpg (deflated 4%)
  adding: content/processed-drone-dataset/train/1/Capture170.jpg (deflated 1%)
  adding: content/processed-drone-dataset/train/1/i (15)-1.jpg (deflated 9%)
  adding: content/processed-drone-dataset/train/1/photo-1470083010285-12169927e64c.jpg (deflated 2%)
  adding: content/processed-drone-dataset/train/1/photo-1488263590619-bc1fff43b6c1.jpg (deflated 2%)
  adding: content/processed-drone-dataset/train/1/Screen-Shot-2017-10-24-at-2.29.18-PM-e1508869812636.jpg (deflated 1%)
  adding: content/processed-drone-dataset/train/1/pexels-photo-1379371.jpg (deflated 3%)
  adding: content/processed-drone-dataset/train/1/Capture14.jpg (deflated 2%)
  adding: content/processed-drone-dataset/train/1/pexels-photo-1852984.jpg (deflated 4%)
  adding: content/processed-drone-dataset/train/1/Capture139.

In [12]:
cp /content/processed-drone-dataset.zip "/content/gdrive/My Drive/processed-drone-dataset.zip"

In [ ]:
cp -r "/content/processed-drone-dataset.zip" "/content/gdrive/My Drive/eva4p2/."

# Create Custom Dataset Class to be used for Test and Train samples

In [13]:
import numpy as np
import cv2
import io
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms
import zipfile
from PIL import Image
import sys
import torch.utils.data
import timeit
import logging

# This is the common class for both test and train data sets. Based on indexes, this class serves the transformed image tensor.
  
#Considered 4 folders (n). Take all the files from those folders into a list, a global list in memory.     
#Flying birds, Large Quadcopters, Small QuadCopters, Winged drones
#Given an index, list[index] will be returned as a file.
#All images are of jpg and 224*224 size.

class WingsDataset(Dataset):

  def __init__(self, root_dir, transform=ToTensor()):
    'Initialization'
    self.m_root = root_dir
    self.m_transform = transform
    logging.basicConfig(filename = "/content/dataset_train_test_timeit.log",datefmt='%H:%M:%S',level=logging.DEBUG)
    #self.g_dict_fileindex = dict()
    self.g_array_fileindex = []
    self.g_array_targetindex = []
    self.walkTheDirectory(self.m_root) 
    self.m_length = len(self.g_array_fileindex)
  
  #root_dir structure
  #train
  #   /folder1
  #     file.jpg
  #     file40.jpg  
  #   /folder2
  #     filex.jpg
  #     filey.40.jpg  
  #   /folder3
  #     filea.jpg
  #     fileb.40.jpg  
  #   /folder4
  #     fileaa.jpg
  #     filebax.jpg  
  
  #test
  #   /anyfoldername1
  #     sfile.jpg
  #     mfile40.jpg  
  #   /foldername2
  #     efilex.jpg
  #     cfiley.40.jpg  
  #   /foldername3
  #     somefilea.jpg
  #     vfileb.40.jpg  
  #   /foldername4
  #     somefileaa.jpg
  #     afilebax.jpg  
  
  
    
  def walkTheDirectory(self,root_dir):
    #for root, dirs, files in os.walk('python/Lib/email'):
    # for file in files:
    #    with open(os.path.join(root, file), "r") as auto:    
    for dir_path, subdirs, files in os.walk(self.m_root): # path should give the complete path till the last directory.
        for filename in files:
            #print(os.path.join(path, filename))
            full_file_path = os.path.join(dir_path, filename)
            #self.g_dict_fileindex[index] = full_path
            self.g_array_fileindex.append(full_file_path)
            dir_name = os.path.basename(dir_path)#get the last directory name from full dir path
            self.g_array_targetindex.append(int(dir_name))

  def __len__(self):
    'Denotes the total number of samples'
    return self.m_length;

  def printItem(self,idx):
    if idx >= self.m_length:
        print('index', idx, 'is bigger than the actual dataset size')
        return
    file = self.g_array_fileindex[idx]
    print('printItem-',file)  

  def __getitem__(self, idx):

    if idx >= self.m_length:
        print('index', idx, 'is bigger than the actual dataset size')
        return
        
    start_time = timeit.default_timer()

    # Pick the file from the index
    filepath = self.g_array_fileindex[idx]
    target = self.g_array_targetindex[idx]

    #already of 224*224 , so dont do any resizing..
    #Apply transformation and give the tensor back
    image = Image.open(filepath).convert('RGB') # even grey scale images will be converted to RGB with 3 channels
    """
    w,h = image.size
    image = image.resize((224,224)) # These are already resized images. Dont do preprocessing here..
    """

    if self.m_transform:
      image = self.m_transform(image)
    
    load_time = timeit.default_timer() - start_time
    #target = torch.Tensor(int(target))
    desc = f' file={filepath} target={target} LOAD_TIME={load_time:0.3f}'
    logging.info(desc)
    return image, target
    

#Train/Test loader

In [14]:

import torch
from torchvision import transforms

#import WingsFolderDataset as wingsDataset
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

train_root_dir = "/content/processed-drone-dataset/train"

# replace with actual mean and std values
#mean= tensor([0.5266, 0.5789, 0.6107])
#std= tensor([0.1701, 0.1641, 0.1792])
normalize = transforms.Normalize(mean=([0.5266, 0.5789, 0.6107]), std=([0.1701, 0.1641, 0.1792]))

train_tr = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop((32,32)),
    transforms.ToTensor(),
    normalize,
    transforms.RandomErasing()
])

train_set = WingsDataset(train_root_dir, transform = train_tr)


train_loader = torch.utils.data.DataLoader(train_set, batch_size=32,
                                          shuffle=True, num_workers=3, pin_memory=False)

#Test loader creation
test_root_dir = "/content/processed-drone-dataset/test"

test_tr = transforms.Compose([transforms.ToTensor(),normalize])
test_set = WingsDataset(test_root_dir, transform = test_tr)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False, num_workers=3)


#Model Used

In [15]:
import torch
import torch.nn.functional as F
import torch.nn as nn

class DroneMobilenetV2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Use ROIpoolng to resize image to 224*224
        #self.ROIpool = nn.AdaptiveMaxPool2d((224,224))
        # Use a pretrained MobilenetV2 model
        self.network = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
        # Replace last layer
        num_ftrs = self.network.classifier[1].in_features
        #Add fully connected layer
        self.network.classifier[1] = nn.Linear(num_ftrs, 4) # output 4 classifications
    
    def forward(self, xb):
        #resized = self.ROIpool(xb)
        out     = self.network(xb)
        return F.log_softmax(out)
#         return torch.sigmoid(self.fc(self.relu(self.network(xb))))

    
    def freeze(self):
        # To freeze the residual layers
        for param in self.network.parameters():
            param.require_grad = False
        for param in self.network.classifier[1].parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.network.parameters():
            param.require_grad = True

# Train, Test utility functions

In [33]:
from __future__ import print_function
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
import copy
from torchsummary import summary
from torchvision import datasets, transforms
import numpy as np

def train_model(model, device, train_loader, optimizer,scheduler, epoch, train_losses, train_acc, criteria, store_mode ='epoch', doL1 = 0, doL2 = 0,LAMBDA = 0):
  print('L1=',doL1,';L2=',doL2,';LAMBDA=',LAMBDA, 'epoch=',epoch)
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  train_loss = 0
    
  #as batches
  #data is the touple returned from get_item
  for batch_idx, (inputs, targets) in enumerate(pbar):
    # get samples
    #data, target = data[0].to(device), target.to(device)
    #print(data_touple)
    #inputs, targets = data_touple
    inputs=inputs.to(device)
    targets=targets.to(device)
    #print('inputs=',inputs.shape,'targets=',targets.shape)
    
    #print('data=',len(data),';target=',len(target))

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(inputs)
    

    # Calculate loss
    #print('y_pred=',len(y_pred.dataset),'target=',len(target.dataset))
    #loss = F.nll_loss(y_pred, target)
    #criteria = nn.CrossEntropyLoss()
    loss = criteria(y_pred, targets) 
    reg_loss=0
    if (doL1 == 1):
      for p in model.parameters():  
        reg_loss += torch.sum(torch.abs(p.data))
    if (doL2 == 1):
      for p in model.parameters():
        reg_loss += torch.sum(p.data.pow(2))    
    
    loss+=LAMBDA*reg_loss
            
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    #after batch, scheduler.step
    scheduler.step()    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(targets.view_as(pred)).sum().item()
    processed += len(inputs)
    train_loss += loss.item()
    """
    if store_mode == 'epoch':   # Store loss and accuracy
        accuracy = 100 * correct / processed
        if not train_losses is None:
            train_losses.append(loss.item())
        if not accuracies is None:
            train_acc.append(accuracy)  
    
    elif  store_mode == 'mini_batch':  # Store loss and accuracy
        batch_accuracy = 100 * correct / processed
        if not train_losses is None:
            train_losses.append(loss.item())
        if not train_acc is None:
            train_acc.append(batch_accuracy)
    """
  train_loss /= len(train_loader.dataset) 
  train_losses.append(train_loss) 
  train_accuracy = 100. * correct / len(train_loader.dataset)
  train_acc.append(train_accuracy)
  print('\n epoch=',epoch,'\nTrain set: Average loss:',train_loss,'Accuracy:',train_accuracy)
    
  #pbar.set_description(desc=  epoch={epoch} f'AverageLoss={loss.item()} Accuracy={100*correct/processed:0.2f}')     
      #pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
  #train_acc.append(100*correct/processed)
    

def test_model(model, device, test_loader,test_losses,test_acc,criteria, correct_samples, incorrect_samples, sample_count=30, last_epoch=False):
    model.eval()
    test_loss = 0
    correct = 0
    #criteria = nn.CrossEntropyLoss()
            
    with torch.no_grad():
        for (inputs, targets) in test_loader:
            #img_batch = data
            #print(targets)
            img_batch = inputs
            #data, target = data.to(device), target.to(device)
            #inputs, targets = data_touple
            inputs=inputs.to(device)
            targets=targets.to(device)
            #print('data=',len(data),';target=',len(target))
            output = model(inputs)
            #test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            #test_loss += criteria(output, target).item()
            test_loss += criteria(output, targets).item()
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            result = pred.eq(targets.view_as(pred))
            """
            if last_epoch:
                #print('last_epoch=',last_epoch)
                for i in range(len(list(result))):
                    if not list(result)[i] and len(incorrect_samples) < sample_count:
                        incorrect_samples.append({
                            'prediction': list(pred)[i],
                            'label': list(targets.view_as(pred))[i],
                            'image': img_batch[i]
                            
                        })
                    elif list(result)[i] and len(correct_samples) < sample_count:
                        correct_samples.append({
                            'prediction': list(pred)[i],
                            'label': list(targets.view_as(pred))[i],
                            'image': img_batch[i]
                            
                        })
            """
            correct += result.sum().item()
            #correct += pred.eq(targets.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 
        100. * correct / len(test_loader.dataset)))
    test_acc.append(100. * correct / len(test_loader.dataset)) 
    return test_loss

#Global functions
def show_summary(model,input_size = (3, 224, 224)):
    summary(model, input_size)
    
def run_model(model, device, optimizer, train_loader, test_loader, train_losses, test_losses, train_acc, test_acc, correct_samples, incorrect_samples, criteria = F.nll_loss, doL1 = 0, doL2 = 0, LAMBDA = 0, EPOCHS = 20,start=0):
    #scheduler = StepLR(optimizer, step_size=8, gamma=0.1)
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1)
    for epoch in range(EPOCHS):
        print("EPOCH:", (start+epoch))
        train_model(model, device, train_loader, optimizer,scheduler, epoch, train_losses, train_acc, criteria, doL1, doL2, LAMBDA)
        last_epoch = False
        if epoch == EPOCHS-1:
          last_epoch = True
        test_model(model, device, test_loader, test_losses, test_acc, criteria,correct_samples,incorrect_samples,last_epoch)
        




# Train the data

In [17]:
import torch.optim as optim

model = DroneMobilenetV2().cuda()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

show_summary(model)

    

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [36]:
train_losses = [] 
train_acc = []
test_losses =[]
test_acc =[]
EPOCHS =25
start =0
optimizer=optim.SGD(model.parameters(), lr=0.003, momentum =0.9, weight_decay=5e-4)
criteria = F.nll_loss
correct_samples = []
incorrect_samples = []
run_model(model, device, optimizer, train_loader, test_loader, train_losses, test_losses, train_acc, test_acc, correct_samples, incorrect_samples, criteria = F.nll_loss, doL1 = 0, doL2 = 0, LAMBDA = 0, EPOCHS = 25,start=0)


  0%|          | 0/354 [00:00<?, ?it/s]

EPOCH: 0
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
100%|██████████| 354/354 [00:12<00:00, 27.96it/s]


 epoch= 0 
Train set: Average loss: 0.03487112579715292 Accuracy: 52.22969385949389



  0%|          | 0/354 [00:00<?, ?it/s]

...


  0%|          | 0/354 [00:00<?, ?it/s]


Test set: Average loss: 0.3849, Accuracy: 1073/4827 (22.23%)

EPOCH: 24
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 24


100%|██████████| 354/354 [00:12<00:00, 27.89it/s]


 epoch= 24 
Train set: Average loss: 0.034620626023165464 Accuracy: 52.672093434790305



Test set: Average loss: 0.1505, Accuracy: 1073/4827 (22.23%)



In [88]:
from __future__ import print_function
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
import copy
from torchsummary import summary
from torchvision import datasets, transforms

def simple_train_model(model, device, train_loader, optimizer,scheduler, epoch,train_losses,train_acc, doL1 = 0,doL2 = 0,LAMBDA = 0):
  print('L1=',doL1,';L2=',doL2,';LAMBDA=',LAMBDA,'epoch=',epoch)
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    #print('y_pred=',len(y_pred.dataset),'target=',len(target.dataset))
    #loss = F.nll_loss(y_pred, target)
    criteria = nn.CrossEntropyLoss()
    loss = criteria(y_pred, target) 
    reg_loss=0
    if (doL1 == 1):
      for p in model.parameters():  
        reg_loss += torch.sum(torch.abs(p.data))
    if (doL2 == 1):
      for p in model.parameters():
        reg_loss += torch.sum(p.data.pow(2))    
    
    loss+=LAMBDA*reg_loss
    
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)
    scheduler.step()

def simple_test_model(model, device, test_loader,test_losses,test_acc):
    model.eval()
    test_loss = 0
    correct = 0
    criteria = nn.CrossEntropyLoss()
            
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            #test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            test_loss += criteria(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def simple_run_model(model, device, optimizer, train_loader, test_loader, train_losses, test_losses, train_acc, test_acc, doL1 = 0, doL2 = 0, LAMBDA = 0, EPOCHS = 20,start=0):
    #scheduler = StepLR(optimizer, step_size=8, gamma=0.1)  
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1)
    for epoch in range(EPOCHS):
        print("EPOCH:", epoch)
        simple_train_model(model, device, train_loader, optimizer,scheduler, epoch, train_losses,train_acc, doL1,doL2,LAMBDA)
        simple_test_model(model, device, test_loader,test_losses,test_acc)



In [89]:
train_losses = [] 
train_acc = []
test_losses =[]
test_acc =[]
EPOCHS =20
start =0
optimizer=optim.SGD(model.parameters(), lr=0.1, momentum =0.9, weight_decay=5e-4)
simple_run_model(model, device, optimizer, train_loader, test_loader, train_losses, test_losses, train_acc, test_acc, doL1 = 0, doL2 = 0, LAMBDA = 0, EPOCHS = 5,start=0)















  0%|          | 0/354 [00:00<?, ?it/s]

EPOCH: 0
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 0


Streaming output truncated to the last 5000 lines.






Loss=0.960879921913147 Batch_id=65 Accuracy=51.94:  18%|█▊        | 64/354 [00:02<00:11, 25.50it/s] 












Loss=1.1164660453796387 Batch_id=66 Accuracy=51.91:  18%|█▊        | 64/354 [00:02<00:11, 25.50it/s]












Loss=1.1164660453796387 Batch_id=66 Accuracy=51.91:  19%|█▉        | 67/354 [00:02<00:11, 25.55it/s]












Loss=1.298292636871338 Batch_id=67 Accuracy=51.79:  19%|█▉        | 67/354 [00:02<00:11, 25.55it/s] 












Loss=1.2387242317199707 Batch_id=68 Accuracy=51.68:  19%|█▉        | 67/354 [00:02<00:11, 25.55it/s]












Loss=1.1080909967422485 Batch_id=69 Accuracy=51.61:  19%|█▉        | 67/354 [00:02<00:11, 25.55it/s]












Loss=1.1080909967422485 Batch_id=69 Accuracy=51.61:  20%|█▉        | 70/354 [00:02<00:11, 25.69it/s]












Loss=1.0709515810012817 Batch_id=70 Accuracy=51.67:  20%|█▉        | 70/354 [00:02<00:11, 25.69it/s]












Loss=1.0243850946426392 Batch_id=71 Acc


Test set: Average loss: 0.0510, Accuracy: 2321/4827 (48.08%)

EPOCH: 1
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 1


Streaming output truncated to the last 5000 lines.






Loss=1.1105455160140991 Batch_id=64 Accuracy=51.39:  18%|█▊        | 64/354 [00:02<00:10, 28.20it/s]












Loss=1.1362403631210327 Batch_id=65 Accuracy=51.28:  18%|█▊        | 64/354 [00:02<00:10, 28.20it/s]












Loss=1.030689001083374 Batch_id=66 Accuracy=51.31:  18%|█▊        | 64/354 [00:02<00:10, 28.20it/s] 












Loss=1.030689001083374 Batch_id=66 Accuracy=51.31:  19%|█▉        | 67/354 [00:02<00:10, 27.51it/s]












Loss=1.2835322618484497 Batch_id=67 Accuracy=51.15:  19%|█▉        | 67/354 [00:02<00:10, 27.51it/s]












Loss=0.8902438282966614 Batch_id=68 Accuracy=51.45:  19%|█▉        | 67/354 [00:02<00:10, 27.51it/s]












Loss=1.2000536918640137 Batch_id=69 Accuracy=51.43:  19%|█▉        | 67/354 [00:02<00:10, 27.51it/s]












Loss=1.2000536918640137 Batch_id=69 Accuracy=51.43:  20%|█▉        | 70/354 [00:02<00:10, 27.00it/s]












Loss=1.2687163352966309 Batch_id=70 Accu

...


Test set: Average loss: 0.1017, Accuracy: 1612/4827 (33.40%)

EPOCH: 3
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 3


Streaming output truncated to the last 5000 lines.






Loss=1.2656608819961548 Batch_id=65 Accuracy=52.46:  18%|█▊        | 64/354 [00:02<00:11, 26.18it/s]












Loss=1.0969477891921997 Batch_id=66 Accuracy=52.38:  18%|█▊        | 64/354 [00:02<00:11, 26.18it/s]












Loss=1.0969477891921997 Batch_id=66 Accuracy=52.38:  19%|█▉        | 67/354 [00:02<00:10, 26.62it/s]












Loss=1.2155847549438477 Batch_id=67 Accuracy=52.25:  19%|█▉        | 67/354 [00:02<00:10, 26.62it/s]












Loss=1.2281750440597534 Batch_id=68 Accuracy=52.22:  19%|█▉        | 67/354 [00:02<00:10, 26.62it/s]












Loss=1.3212604522705078 Batch_id=69 Accuracy=51.96:  19%|█▉        | 67/354 [00:02<00:10, 26.62it/s]












Loss=1.3212604522705078 Batch_id=69 Accuracy=51.96:  20%|█▉        | 70/354 [00:02<00:10, 26.88it/s]












Loss=1.092908263206482 Batch_id=70 Accuracy=51.98:  20%|█▉        | 70/354 [00:02<00:10, 26.88it/s] 












Loss=1.0251903533935547 Batch_id=71 Acc


Test set: Average loss: 0.0572, Accuracy: 2013/4827 (41.70%)

EPOCH: 4
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 4


Streaming output truncated to the last 5000 lines.






Loss=1.1449580192565918 Batch_id=63 Accuracy=51.12:  18%|█▊        | 63/354 [00:02<00:10, 27.34it/s]












Loss=0.990390419960022 Batch_id=64 Accuracy=51.35:  18%|█▊        | 63/354 [00:02<00:10, 27.34it/s] 












Loss=1.1807032823562622 Batch_id=65 Accuracy=51.33:  18%|█▊        | 63/354 [00:02<00:10, 27.34it/s]












Loss=1.1807032823562622 Batch_id=65 Accuracy=51.33:  19%|█▊        | 66/354 [00:02<00:10, 27.01it/s]












Loss=1.216484785079956 Batch_id=66 Accuracy=51.17:  19%|█▊        | 66/354 [00:02<00:10, 27.01it/s] 












Loss=1.0142048597335815 Batch_id=67 Accuracy=51.19:  19%|█▊        | 66/354 [00:02<00:10, 27.01it/s]












Loss=1.2424499988555908 Batch_id=68 Accuracy=51.04:  19%|█▊        | 66/354 [00:02<00:10, 27.01it/s]












Loss=1.2424499988555908 Batch_id=68 Accuracy=51.04:  19%|█▉        | 69/354 [00:02<00:10, 26.80it/s]












Loss=1.1416727304458618 Batch_id=69 Acc


Test set: Average loss: 0.0452, Accuracy: 2006/4827 (41.56%)



###The below content is a copy paste about mobilenet information..
### This notebook is optionally accelerated with a GPU runtime.
### If you would like to use this acceleration, please select the menu option "Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and click "SAVE"

----------------------------------------------------------------------

# MobileNet v2

*Author: Pytorch Team*

**Efficient networks optimized for speed and memory, with residual blocks**

_ | _
- | -
![alt](https://pytorch.org/assets/images/mobilenet_v2_1.png) | ![alt](https://pytorch.org/assets/images/mobilenet_v2_2.png)

In [ ]:
import torch
#define the model
model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
model.eval()

In [ ]:
#trace model with a dummy input
traced_model = torch.jit.trace(model, torch.randn(1,3,224,224))
traced_model.save('mobilenetV2.pt')

All pre-trained models expect input images normalized in the same way,
i.e. mini-batches of 3-channel RGB images of shape `(3 x H x W)`, where `H` and `W` are expected to be at least `224`.
The images have to be loaded in to a range of `[0, 1]` and then normalized using `mean = [0.485, 0.456, 0.406]`
and `std = [0.229, 0.224, 0.225]`.

Here's a sample execution.

In [ ]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
print(torch.nn.functional.softmax(output[0], dim=0))


### Model Description

The MobileNet v2 architecture is based on an inverted residual structure where the input and output of the residual block are thin bottleneck layers opposite to traditional residual models which use expanded representations in the input. MobileNet v2 uses lightweight depthwise convolutions to filter features in the intermediate expansion layer. Additionally, non-linearities in the narrow layers were removed in order to maintain representational power.

| Model structure | Top-1 error | Top-5 error |
| --------------- | ----------- | ----------- |
|  mobilenet_v2       | 28.12       | 9.71       |


### References

 - [MobileNetV2: Inverted Residuals and Linear Bottlenecks](https://arxiv.org/abs/1801.04381)